In [1]:
!pip install tensorflow
!pip install opencv-python

     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ---------------------------------------- 57.5/57.5 kB ? eta 0:00:00
     ---------------------------------------- 0.0/65.5 kB ? eta -:--:--
     ---------------------------------------- 65.5/65.5 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     ----- ---------------------------------- 0.2/1.5 MB 12.2 MB/s eta 0:00:01
     -------------- ------------------------- 0.5/1.5 MB 8.3 MB/s eta 0:00:01
     ------------------------ --------------- 0.9/1.5 MB 8.3 MB/s eta 0:00:01
     ----------------------------------- ---- 1.3/1.5 MB 7.6 MB/s eta 0:00:01
     ---------------------------------------- 1.5/1.5 MB 7.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/181.3 kB ? eta -:--:--
     ------------------------------------- 181.3/181.3 kB 11.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
     ----------

In [3]:
!pip install scikit-learn

     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.4 kB ? eta -:--:--
     ---------------------------------------- 60.4/60.4 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.6 MB 10.6 MB/s eta 0:00:01
   --- ------------------------------------ 0.9/10.6 MB 10.8 MB/s eta 0:00:01
   ---- ----------------------------------- 1.2/10.6 MB 11.0 MB/s eta 0:00:01
   ------- -------------------------------- 1.9/10.6 MB 10.8 MB/s eta 0:00:01
   -------- ------------------------------- 2.4/10.6 MB 10.8 MB/s eta 0:00:01
   ---------- ----------------------------- 2.9/10.6 MB 10.9 MB/s eta 0:00:01
   ------------- -------------------------- 3.5/10.6 MB 11.0 MB/s eta 0:00:01
   --------------- ------------------------ 4.0/10.6 MB 11.1 MB/s eta 0:00:01
   ----------------- ---------------------- 4.5/10.6 MB 11.1 MB/s eta 0:00:01
   

In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Function to load video frames
def load_video_frames(video_path, max_frames=148, resize_shape=(128, 128)):
    frames = []
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened() and len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        # Resize each frame for consistency
        frame = cv2.resize(frame, resize_shape)
        frames.append(frame)

    cap.release()
    
    if not frames:
        # If no frames were loaded, add a placeholder frame
        frames.append(np.zeros((resize_shape[0], resize_shape[1], 3), dtype=np.uint8))

    # Pad or trim frames to ensure a consistent length
    frames = frames + [np.zeros_like(frames[0])] * (max_frames - len(frames))
    return np.array(frames)


# Generator function to load dataset in batches
def data_generator(folder, max_frames_per_video=148, batch_size=32):
    video_files = os.listdir(folder)

    while True:
        batch_files = np.random.choice(video_files, size=batch_size)
        batch_videos = [load_video_frames(os.path.join(folder, file), max_frames_per_video) for file in batch_files]
        batch_labels = [1 if "real" in file else 0 for file in batch_files]

        yield np.array(batch_videos), np.array(batch_labels)

# Function to load dataset in smaller chunks
def load_dataset_chunked(folder, max_frames_per_video=148, chunk_size=50):
    videos = []
    labels = []

    video_files = os.listdir(folder)
    for i in range(0, len(video_files), chunk_size):
        chunk_files = video_files[i:i+chunk_size]
        chunk_videos = [load_video_frames(os.path.join(folder, file), max_frames_per_video) for file in chunk_files]
        chunk_labels = [1 if "real" in file else 0 for file in chunk_files]

        videos.extend(chunk_videos)
        labels.extend(chunk_labels)

    return np.array(videos), np.array(labels)

In [5]:
# Load dataset
real_folder = "Celeb_real_face_only"
fake_folder = "Celeb_fake_face_only"
X, y = load_dataset_chunked(real_folder, max_frames_per_video=148, chunk_size=50)

In [6]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Convert NumPy arrays to TensorFlow tensors
X_train_tf = tf.ragged.constant(X_train, dtype=tf.float32)
X_test_tf = tf.ragged.constant(X_test, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_test_tf = tf.convert_to_tensor(y_test, dtype=tf.float32)